In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. تحميل البيانات
def load_data(image_folder, img_size=(64, 64)):
    classes = {"Drowsy": 1, "Non Drowsy": 0}
    X, y = [], []
    
    for class_name in classes:
        class_dir = os.path.join(image_folder, class_name)
        images = os.listdir(class_dir)
        for img_name in images:
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(classes[class_name])
    
    X = np.array(X)
    y = to_categorical(y)
    return X, y

dataset_path = "/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)"
X, y = load_data(dataset_path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 2. Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)
datagen.fit(X_train)

# 3. بناء النموذج
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 4. تدريب النموذج
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop, checkpoint]
)

# 5. تقييم النموذج
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,625,410 (6.20 MB)

 Trainable params: 1,625,410 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2090/2090 ━━━━━━━━━━━━━━━━━━━━ 138s 65ms/step - accuracy: 0.6856 - loss: 3.8074 - val_accuracy: 0.8550 - val_loss: 0.2928
Epoch 2/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 132s 63ms/step - accuracy: 0.8898 - loss: 0.2460 - val_accuracy: 0.9675 - val_loss: 0.0807
Epoch 3/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 131s 63ms/step - accuracy: 0.9492 - loss: 0.1361 - val_accuracy: 0.9908 - val_loss: 0.0314
Epoch 4/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 132s 63ms/step - accuracy: 0.9666 - loss: 0.0990 - val_accuracy: 0.9883 - val_loss: 0.0474
Epoch 5/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 130s 62ms/step - accuracy: 0.9760 - loss: 0.0861 - val_accuracy: 0.9971 - val_loss: 0.0096
Epoch 6/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 142s 68ms/step - accuracy: 0.9841 - loss: 0.0605 - val_accuracy: 0.9762 - val_loss: 0.0592
Epoch 7/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 142s 68ms/step - accuracy: 0.9856 - loss: 0.0476 - val_accuracy: 0.9920 - val_loss: 0.0239
Epoch 8/20
2090/2090 ━━━━━━━━━━━━━━━━━━━━ 131s 62ms/step - accuracy: 0.9871 - lo

In [11]:
model.save('/kaggle/working/driver_drowsiness_model.h5')

In [12]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/kaggle/working/driver_drowsiness_model.h5')
loss, accuracy = loaded_model.evaluate(X_test, y_test)
print(f"Loaded Model Test Accuracy: {accuracy*100:.2f}%")

262/262 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.9970 - loss: 0.0149
Loaded Model Test Accuracy: 99.75%


In [13]:
import cv2
import numpy as np

img_path = "/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/Drowsy/A0010.png"  
img = cv2.imread(img_path)
img = cv2.resize(img, (64, 64)) 
img = np.expand_dims(img, axis=0) 

prediction = loaded_model.predict(img)
class_names = ["Non Drowsy", "Drowsy"]
predicted_class = class_names[np.argmax(prediction)]
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Predicted class: Drowsy


In [15]:
img_path = "/kaggle/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/Non Drowsy/a0098.png" 
img = cv2.imread(img_path)
img = cv2.resize(img, (64, 64))  
img = np.expand_dims(img, axis=0)  

prediction = loaded_model.predict(img)
class_names = ["Non Drowsy", "Drowsy"]
predicted_class = class_names[np.argmax(prediction)]
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted class: Non Drowsy


In [16]:
img_path = "/kaggle/input/hellllo/618abc9cdfbae01bc36d2ce5_hero_advanced_drowsiness.jpg" 
img = cv2.imread(img_path)
img = cv2.resize(img, (64, 64))
img = np.expand_dims(img, axis=0)  

prediction = loaded_model.predict(img)
class_names = ["Non Drowsy", "Drowsy"]
predicted_class = class_names[np.argmax(prediction)]
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted class: Drowsy
